In [1]:
from langchain.retrievers.parent_document_retriever import MultiVectorRetriever

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader

In [3]:
loaders = [
    TextLoader('../../paul_graham_essay.txt'),
    TextLoader('../../state_of_the_union.txt'),
]
docs = []
for l in loaders:
    docs.extend(l.load())

In [4]:
# This text splitter is used to create the child documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Write a summary of the following document: {doc}"
)

runnable = RunnableMap({
    "doc": lambda x: x.page_content
}) | prompt | ChatOpenAI() | StrOutputParser()

In [11]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore, 
    docstore=store, 
    operators=[runnable]
)

In [12]:
docs = parent_splitter.split_documents(docs)

In [13]:
retriever.add_documents(docs, ids=None)

In [15]:
sub_docs = vectorstore.similarity_search("justice breyer")

In [16]:
sub_docs[0]

Document(page_content='The document highlights the passing of new laws in various states that aim to suppress voting and undermine elections. The author calls on the Senate to pass three acts: the Freedom to Vote Act, the John Lewis Voting Rights Act, and the Disclose Act, which aims to reveal the funding sources of elections. The author also pays tribute to Justice Stephen Breyer for his service and announces the nomination of Judge Ketanji Brown Jackson to the United States Supreme Court. The document emphasizes the importance of securing the border and fixing the immigration system, highlighting various measures that have been taken, such as implementing new technology, joint patrols, and establishing dedicated immigration judges. Additionally, the document mentions the support provided to South and Central American countries to host refugees and secure their borders.', metadata={'doc_id': '9630e034-8ecc-4551-b6d0-a9ceaca28726'})

In [19]:
retrieved_docs = retriever.get_relevant_documents("justice breyer")

In [20]:
retrieved_docs[0]

Document(page_content='In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. \n\nWe cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in priva